In [ ]:
import pandas as pd, matplotlib.pyplot as plt, numpy as np, seaborn as sns, os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cluster import KMeans
import joblib

## Extração de dados simulados

In [ ]:
df_cicle = pd.DataFrame()

df = pd.read_csv(r"C:\Users\Davi\Documents\Projetos\FIAP\FASE 4\Enterprise Challenge\config\sensor_data_30000.csv")
df['timestamp_ms'] = df['timestamp_ms']/1000
df["acc_total"] = np.sqrt(df["accX_g"]**2 + df["accY_g"]**2 + df["accZ_g"]**2)

display(df_cicle)
display(df)

## Divisão de dados de treino e dados de teste 70 - 30


In [ ]:
X = df.loc[df['cycleCount']>=4200, ['tempC', 'currentA', 'acc_total']]
y = df.loc[df['cycleCount']>=4200,'cycleCount']

df_ml = pd.DataFrame({'cycle':df.loc[df['cycleCount']<4200,'cycleCount'], 'tempC':df.loc[df['cycleCount']<4200,'tempC'], 'currentA':df.loc[df['cycleCount']<4200,'currentA'], 'acc_total':df.loc[df['cycleCount']<4200,'acc_total']})
print(df_ml)
# x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.3, random_state=30)
# print(f'x_train: {x_train.shape}\nx_test: {x_test.shape}\ny_train: {y_train.shape}\ny_test: {y_test.shape}')
# print(x_train, y_train)

## Deteção de anomalias
Com esse modelo, é possivel identificar comportamentos fora do padrão 

-1: Anormal

1: Normal

In [ ]:
# X = dados combinando temperatura, corrente e vibração
for col in ['tempC', 'currentA', 'acc_total']:
    model = IsolationForest(contamination=0.01)
    model.fit(X[[col]])

    df_ml[f'{col}_anomaly'] = model.predict(df_ml[[col]])

### Gerando visual para identificação de anomalias
Os dados serão gerados com base em um perido de 10 minutos (120 ciclos)

Os pontos azuis representam os dados considerados sem anomalias, enquanto os pontos vermelhos representam as anomalias encontradas.

In [ ]:
for col in ['tempC', 'currentA', 'acc_total']:
    plt.figure(figsize=(12, 4))
    sns.scatterplot(data=df_ml.loc[df_ml['cycle']<=120], x='cycle', y=col, hue=f'{col}_anomaly', palette={1: 'blue', -1: 'red'})
    plt.title(f'Anomalias detectadas na variável {col}')
    plt.legend(title='Anomalia')
    plt.xlabel('Ciclo')
    plt.ylabel(col)
    plt.grid(True)
    plt.show()